In [1]:
import pandas as pd
import numpy as np

In [2]:
# Custom conversion function to convert numerical strings to floats.
# For strings starting with the '<' sign, just divide the number by 10 to 
# make it much smaller than the rest of the dataset.
def custom_convert(val):
    if isinstance(val, str) and val.strip().startswith('<'):
        numstr = val.split('<')[1]
        val=float(numstr)
        return(val/10)
    try:
        return float(val)
    except ValueError:
        return None  # or np.nan if you prefer


def convert_strings_to_float(df):
    custom_convert_columns = ['particulate_microcystin', 'dissolved_microcystin', 'total_dissolved_p',
                             'extracted_phycocyanin', 'soluble_reactive_p', 'extracted_chla', 'ammonia', 'nitrate_nitrite']

    # Apply to columns that have datatype as object
    for column in custom_convert_columns:
        if column in df.columns:
            df[column] = df[column].apply(custom_convert)
            
    return df

In [3]:
def rename_columns_to_match_2021(df):
    dflocal = df.rename(columns={'Date': 'date',
                            'Site': 'station_name',
                            'Local Time (Eastern Time Zone)': 'time',
                            'Latitude (decimal deg)': 'lat', 
                            'Longitude (decimal deg)': 'lon',
                            'Particulate Microcystin (µg/L)': 'particulate_microcystin',
                            'Dissolved Microcystin (µg/L)': 'dissolved_microcystin',
                            'Extracted Phycocyanin (µg/L)': 'extracted_phycocyanin',
                            'Extracted Chlorophyll a (µg/L)': 'extracted_chla',
                            'Turbidity (NTU)': 'turbidity',
                            'Total Suspended Solids (mg/L)': 'tss',
                            'Volatile Suspended Solids (mg/L)': 'vss',
                            'Total Phosphorus (µg P/L)': 'total_p',
                            'Total Dissolved Phosphorus (µg P/L)': 'total_dissolved_p',
                            'Soluble Reactive Phosphorus (µg P/L)': 'soluble_reactive_p',
                            'Ammonia (µg N/L)': 'ammonia',
                            'Nitrate + Nitrite (mg N/L)': 'nitrate_nitrite',
                            'Particulate Organic Carbon (mg/L)': 'particulate_organic_c',
                            'Particulate Organic Nitrogen (mg/L)': 'particulate_organic_n',
                            'Colored Dissolved Organic Material absorbance (m-1) at 400nm ': 'cdom',
                           })
    return dflocal

In [4]:
def handle_missing_lat_lon(df):
    station_dict = {
    "WE2": (41.762,-83.33), 
    "WE4": (41.827,-83.193),
    "WE6": (41.705,-83.385),
    "WE8": (41.834,-83.364),
    "WE9": (41.718,-83.424),
    "WE12": (41.703,-83.254),
    "WE13": (41.741,-83.136),
    "WE16": (41.66,-83.143)
    }
    
    # Replace empty strings with NA for consistent missing handling
#    df['lat'].replace('', np.nan, inplace=True)
#    df['lon'].replace('', np.nan, inplace=True)
    df['lat'] = df['lat'].replace('', np.nan)
    df['lon'] = df['lon'].replace('', np.nan)


    # Apply imputation: fill in missing lat/lon using the dictionary
    df['lat'] = df.apply(
        lambda row: station_dict[row['station_name']][0]
        if pd.isna(row['lat']) and row['station_name'] in station_dict else row['lat'],
        axis=1
    )

    df['lon'] = df.apply(
        lambda row: station_dict[row['station_name']][1]
        if pd.isna(row['lon']) and row['station_name'] in station_dict else row['lon'],
        axis=1
    )

    return df

In [5]:
def handle_timestamp_and_geocoordinates(df):
    df['lat'] = df['lat'].astype(float)
    df['lon'] = df['lon'].astype(float)

    # Combine and convert to datetime64
    df['time'] = df['time'].fillna('00:00')
    df['timestamp'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    #df = df.drop(columns=['date', 'time'])
    return df


In [6]:
def extract_subset_columns(df):
    subset = ['station_name', 'timestamp', 'lat', 'lon', 
              'particulate_microcystin', 'dissolved_microcystin',
              'extracted_phycocyanin', 'extracted_chla',
              'turbidity', 'tss', 'vss', 'cdom', 'date']

    df = df[subset]
    return df

In [7]:

filelist = [
    './0187718/2.2/data/0-data/lake_erie_habs_field_sampling_results_2012_2018_v2.csv',
    './0209116/1.1/data/0-data/lake_erie_habs_field_sampling_results_2019.csv',
    './0254720/1.1/data/0-data/noaa-glerl-erie-habs-field-sampling-results-2020-2021.csv'
]

multiyear_df = pd.DataFrame()

for file in filelist:
    df = pd.read_csv(file, encoding='latin1')
    df = rename_columns_to_match_2021(df)
    df = handle_timestamp_and_geocoordinates(df)
    df = convert_strings_to_float(df)
    df = extract_subset_columns(df)
    df = handle_missing_lat_lon(df)
    multiyear_df = pd.concat([multiyear_df, df], ignore_index=True)


In [8]:
multiyear_df

,station_name,timestamp,lat,lon,particulate_microcystin,dissolved_microcystin,extracted_phycocyanin,extracted_chla,turbidity,tss,vss,cdom,date
0,WE2,2012-05-15 10:40:00,41.0127,-83.0055,0.01,NaN,0.28,3.67,NaN,3.47,1.11,NaN,5/15/2012
1,WE4,2012-05-15 10:08:00,41.8267,-83.1933,0.01,NaN,0.17,3.05,NaN,3.15,1.01,NaN,5/15/2012
2,WE6,2012-05-15 11:02:00,41.7052,-83.3853,0.01,NaN,0.45,2.24,NaN,2.90,0.98,NaN,5/15/2012
3,WE8,2012-05-15 11:31:00,41.8339,-83.3640,0.01,NaN,0.52,5.73,NaN,4.32,1.06,NaN,5/15/2012
4,WE2,2012-05-31 11:15:00,41.7625,-83.3286,0.01,NaN,0.61,6.07,NaN,21.42,2.4,NaN,5/31/2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,WE2,2021-10-27 08:50:00,41.7629,-83.3305,0.13,0.13,1.85,9.49,12.3,16.12,1.96,1.32,10/27/2021
1872,WE2,2021-10-27 00:00:00,41.7620,-83.3300,0.01,0.13,1.15,8.74,NaN,NaN,NaN,NaN,10/27/2021
1873,WE6,2021-10-27 09:16:00,41.7067,-83.3840,0.01,0.11,0.73,2.25,34.0,38.40,4,7.01,10/27/2021
1874,WE8,2021-10-27 08:26:00,41.8330,-83.3614,0.01,0.16,1.48,7.34,5.4,6.44,1.08,2.13,10/27/2021


In [9]:
multiyear_df[(multiyear_df['timestamp'] > '2012-01-01') & \
             (multiyear_df['dissolved_microcystin'] > 0.3)]

,station_name,timestamp,lat,lon,particulate_microcystin,dissolved_microcystin,extracted_phycocyanin,extracted_chla,turbidity,tss,vss,cdom,date
234,WE2,2014-09-08 09:51:00,41.7652,-83.3326,0.60,0.48,3.24,20.77,9.23,10.78,3.74,0.68,9/8/2014
236,WE6,2014-09-08 10:28:00,41.7108,-83.3813,2.10,1.94,15.32,53.38,26.20,29.80,13.05,1.46,9/8/2014
237,WE8,2014-09-08 13:13:00,41.8334,-83.3636,3.50,0.91,31.94,41.82,17.60,20.60,9.9,1.23,9/8/2014
238,WE12,2014-09-08 11:08:00,41.7044,-83.2554,0.40,0.31,4.44,23.20,9.46,12.47,5.47,0.66,9/8/2014
240,WE2,2014-09-15 09:51:00,41.7636,-83.3310,0.70,0.87,9.10,13.72,10.20,11.46,2.58,0.79,9/15/2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829,WE2,2021-09-27 14:11:00,41.7615,-83.3320,0.53,0.37,12.47,30.20,9.95,11.88,3.72,2.11,9/27/2021
1833,WE8,2021-09-27 09:39:00,41.8321,-83.3595,0.67,0.33,3.31,27.81,9.92,13.16,3.56,1.20,9/27/2021
1834,WE8,2021-09-27 00:00:00,41.8340,-83.3640,0.45,0.36,3.60,27.40,NaN,NaN,NaN,NaN,9/27/2021
1836,WE12,2021-09-27 11:16:00,41.7046,-83.2575,0.44,0.36,2.58,29.08,14.30,15.24,4.4,0.98,9/27/2021


In [10]:
multiyear_df.to_csv("./glrl-hab-data.csv")